In [31]:
import pandas as pd
import numpy as np

In [32]:
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split

In [33]:
gpu = pd.read_csv('gpu.csv')

In [34]:
gpu.describe()

,releaseYear,memSize,memBusWidth,gpuClock,memClock,tmu,rop,G3Dmark,G2Dmark,testDate
count,786.000000,786.000000,786.000000,786.000000,786.000000,786.000000,786.000000,786.000000,786.000000,786.000000
mean,2011.944020,3.078112,242.198473,783.315522,981.801527,57.486005,22.952926,3207.436387,381.519084,2013.452926
std,4.675391,4.748925,454.525419,341.528822,431.322303,65.490451,23.245115,5000.391155,261.783492,3.693816
min,2000.000000,0.032000,32.000000,143.000000,166.000000,2.000000,1.000000,1.000000,21.000000,2009.000000
25%,2008.000000,0.512000,128.000000,550.000000,750.000000,16.000000,8.000000,264.500000,166.250000,2010.000000
50%,2012.000000,2.000000,128.000000,725.000000,900.000000,32.000000,16.000000,1007.000000,319.000000,2013.000000
75%,2015.000000,4.000000,256.000000,970.000000,1250.000000,80.000000,32.000000,3556.750000,555.000000,2016.000000
max,2022.000000,48.000000,4096.000000,2321.000000,2248.000000,336.000000,128.000000,29094.000000,1117.000000,2022.000000


In [35]:
X = gpu.drop(columns = ['G3Dmark'])

In [36]:
X.head()

,manufacturer,GPU,releaseYear,memSize,memBusWidth,gpuClock,memClock,tmu,rop,igp,bus,memType,gpuChip,G2Dmark,testDate,category
0,NVIDIA,GeForce MX550,2022,2.0,64.0,1065,1500.0,32,16,No,PCIe 4.0 x8,GDDR6,TU117,424,2022,Unknown
1,NVIDIA,GeForce RTX 3090 Ti,2022,24.0,384.0,1560,1313.0,336,112,No,PCIe 4.0 x16,GDDR6X,GA102,1117,2022,Unknown
2,AMD,Radeon RX 6400,2022,4.0,64.0,1923,2000.0,48,32,No,PCIe 4.0 x4,GDDR6,Navi 24,899,2022,Unknown
3,AMD,Radeon RX 6500 XT,2022,4.0,64.0,2310,2248.0,64,32,No,PCIe 4.0 x4,GDDR6,Navi 24,910,2022,Unknown
4,AMD,Radeon RX 6700S,2022,8.0,128.0,1700,1750.0,112,64,No,PCIe 4.0 x8,GDDR6,Navi 23,829,2022,Unknown


In [37]:
y = gpu.G3Dmark

In [38]:
y.head(), y.shape

(0     5090
 1    29094
 2     6958
 3     9445
 4    11989
 Name: G3Dmark, dtype: int64,
 (786,))

In [39]:
X.dtypes

manufacturer     object
GPU              object
releaseYear       int64
memSize         float64
memBusWidth     float64
gpuClock          int64
memClock        float64
tmu               int64
rop               int64
igp              object
bus              object
memType          object
gpuChip          object
G2Dmark           int64
testDate          int64
category         object
dtype: object

So I've split my data into X and y. Y is of course my predictor variable, the 3DMark scores for every card, and X is all my other columns. Since 7 of my columns in X are categorical, I need to create dummies by one hot encoding them so they can be used in my model. Rememebr, I'll be creating a random forrest moodel here. Since I'll be using a random forest model, I dont neccesarily need to scale my data. (Hey Steve, I know the rubric says we need to scale the data, bit I'll try first with no scaling since we're using a tree based model)

I know were getting a bit ahead, but my process will be...

Create Dummies
Split my data
Create a model instance
Go from there.

Let's creat the dummies. 

In [40]:
X = pd.get_dummies(X)

In [41]:
X.head()

,releaseYear,memSize,memBusWidth,gpuClock,memClock,tmu,rop,G2Dmark,testDate,manufacturer_AMD,...,gpuChip_Vega 20,gpuChip_Venus,gpuChip_Vesuvius,gpuChip_Whistler,gpuChip_Wimbledon,category_Desktop,category_Mobile,"category_Mobile, Workstation",category_Unknown,category_Workstation
0,2022,2.0,64.0,1065,1500.0,32,16,424,2022,False,...,False,False,False,False,False,False,False,False,True,False
1,2022,24.0,384.0,1560,1313.0,336,112,1117,2022,False,...,False,False,False,False,False,False,False,False,True,False
2,2022,4.0,64.0,1923,2000.0,48,32,899,2022,True,...,False,False,False,False,False,False,False,False,True,False
3,2022,4.0,64.0,2310,2248.0,64,32,910,2022,True,...,False,False,False,False,False,False,False,False,True,False
4,2022,8.0,128.0,1700,1750.0,112,64,829,2022,True,...,False,False,False,False,False,False,False,False,True,False


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)

In [43]:
X_train.shape, X_test.shape

((589, 985), (197, 985))

In [44]:
y_train.shape, y_test.shape

((589,), (197,))

In [45]:
X_train.dtypes

releaseYear                       int64
memSize                         float64
memBusWidth                     float64
gpuClock                          int64
memClock                        float64
                                 ...   
category_Desktop                   bool
category_Mobile                    bool
category_Mobile, Workstation       bool
category_Unknown                   bool
category_Workstation               bool
Length: 985, dtype: object

Will the one hot features here being bools create issues? 